# Bernstein-Vazirani

In [ ]:
from qat.lang.AQASM import *

In [ ]:
@build_gate("MyOracle", [int, str])
def bitwise_product(nqubits, bitstring):
    qrout = QRoutine()
    qreg = qrout.new_wires(nqubits)
    qout = qrout.new_wires(1)
    for bit, qbit in zip(bitstring, qreg):
        if bit == "1":
            qrout.apply(CNOT, qbit, qout)
        elif bit != "0":
            raise Exception("In a bitsring only 0's and 1's are admissible")
    return qrout
            

Same program as Deutsch-Jozsa

We first generate a random bitstring

In [ ]:
nqubits = 5

In [ ]:
import random
bitstring = bin(random.getrandbits(nqubits))[2:].zfill(nqubits)

print(bitstring)

In [ ]:

# Previously we used an abstract gate
# my_oracle = AbstractGate("MyOracle", [int])

pr = Program()
qr = pr.qalloc(nqubits)
qout = pr.qalloc(1)

pr.apply(X, qout)
pr.apply(H, qout)

for qb in qr:
    pr.apply(H, qb)
# now we provide the implementation of the oracle directly
pr.apply(bitwise_product(nqubits, bitstring), [*qr, *qout])
for qb in qr:
    pr.apply(H, qb)
pr.apply(H, qout)
pr.apply(X, qout)

In [ ]:
circ = pr.to_circ()
%qatdisplay circ

If we want to display the whole circuit, we can pass the --depth parameter

In [ ]:
%qatdisplay circ --depth 2

Having provided the right implementation, we can simulate without the additional linking stage

In [ ]:
from qat.qpus import PyLinalg
qpu = PyLinalg()
res = qpu.submit(circ.to_job())
for sample in res:
    print(f"State {sample.state} with amplitude {sample.amplitude}")